In [51]:
#create compute cluster
from azureml.core.compute import AmlCompute
from azureml.pipeline.core import PipelineData, Pipeline
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace,Dataset, Datastore
#import conda dependencies
from azureml.core.conda_dependencies import CondaDependencies
#import experiment
from azureml.core.experiment import Experiment
#import environment
from azureml.core.environment import Environment


In [52]:
ws=Workspace.from_config('C:/Users/satya/Downloads/Azure/Azure_cp/.azureml/config12.json')

In [ ]:
envt=Environment(name="myenv")
myenvdep=CondaDependencies.create(conda_packages=['pandas','scikit-learn'])
envt.python.conda_dependencies=myenvdep
envt.register(ws)

In [54]:
cluster_name = 'amlcompute12'
config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=1)
compute = AmlCompute.create(ws, cluster_name, config)

In [56]:
input_ds=Dataset.get_by_name(ws,'credit_dataset')

In [57]:
#list registered datasets in workspace
list(ws.datasets.keys())

['credit_dataset']

In [42]:
print(input_ds)

None


In [58]:
runconfig=RunConfiguration()
runconfig.target=compute
runconfig.environment=envt

In [59]:
#import pipeline
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

In [60]:
datafolder=PipelineData(name='datafolder',datastore=Datastore.get(ws,'credit_datastore'))

In [65]:
dataprep=PythonScriptStep(name='dataprep',
                        source_directory='.', script_name='dataprep.py',
                        inputs=[input_ds.as_named_input('raw_data')],
                        outputs=[datafolder],
                        runconfig=runconfig,
                        arguments=['--datafolder',datafolder])

trainstep=PythonScriptStep(name='train',
                        source_directory='.',
                        script_name='train.py',
                        inputs=[datafolder],
                        runconfig=runconfig,
                        arguments=['--datafolder',datafolder])

In [ ]:
steps=[dataprep,trainstep]
pipeline=Pipeline(workspace=ws,steps=steps)


In [ ]:
new_exp=Experiment(ws,'credit_pipeline')
new_exp.submit(pipeline)
new_exp.wait_for_completion(show_output=True)